In [2]:
"""
author: Miranda Lv
Date: Aug, 2023
"""

In [25]:
import rasterio
import os
from os.path import dirname as up
import glob
from pathlib import Path
from rasterio.plot import show
import geopandas as gpd

In [26]:
root_path = up(os.path.abspath('.'))
tiffDir = os.path.join(root_path, 'data/output')
aoi = os.path.join(root_path, 'data/processing_data/vectors/subregion_aoi_wgs84.geojson')

In [27]:
aio_gdf = gpd.read_file(aoi)
aio_gdf = aio_gdf.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [18]:
alltif = list(Path(dataDir).glob("*.tif"))

In [19]:
test = alltif[0]

In [22]:
src = rasterio.open(test)

In [ ]:
def Clipper(raster, vector):


    # Read Shapefile
    with fiona.open(vector, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rio.open(raster) as src:
        # read imagery file
        out_image, out_transform = mask.mask(src, shapes, crop=True, nodata=np.nan)

        # Check that after the clip, the image is not empty
        test = out_image[~np.isnan(out_image)]

        if test[test > 0].shape[0] == 0:
            raise RuntimeError("Empty output")

        out_meta = src.profile
        out_meta.update({"height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

    return (out_image, out_meta)


# Clip the raster
array_out, out_meta = Clipper(raster, path)

# Save the clip as a tif
with rio.open(clipped, "w", **out_meta) as dest:
    dest.write(array_out)